In [ ]:
"""
.. _example-usage:

Simple Step-By-Step Example on How to do ML with torchpme
=========================================================

.. currentmodule:: torchpme

This example showcases how the main capabilities of ``torchpme``

"""


In [3]:
import numpy as np
import ase.io
import chemiscope
from ase import Atoms
import matplotlib.pyplot as plt
import torch
from vesin.torch import NeighborList

from torchpme import CombinedPotential, EwaldCalculator, InversePowerLawPotential
from torchpme.utils.prefactors import eV_A
from torchpme.potentials import CoulombPotential


# 

# Generate Simple Example Structures
Throughout this tutorial, we will work with a simple atomic structure in three dimensions, which is a distorted version of the CsCl structure.

In [ ]:
# Generate a single unit cell of CsCl (2 atoms in the cell)
structure_single = Atoms('CsCl', cell=np.eye(3), positions=[[0,0,0],[0.5,0.5,0.5]])

# Generate a big structure by periodically repeating the unit cell and apply a random
# distortion to all atoms
structure_big = structure_single.repeat(3)
structure_big.rattle(stdev=0.01)
structure_big.wrap() # make sure all atoms are in the unit cell

# Get the required variables for later use and define the charges for each atom:
# all Cs atoms get a charge of +1, all Cl atoms a charge of -1
positions = torch.tensor(structure_big.positions)
cell = torch.tensor(structure_big.cell)
num_atoms = len(positions)
charges = torch.ones((num_atoms,))
charges[::2] *= -1

# Define Potential
We now need to define the potential function with which the atoms interact. Since this is a library for long-range ML, we support three major options:
1. the Coulomb potential ($1/r$)
2. more general inverse power-law potentials ($1/r^p$)
3. an option to build custom potentials using splines

This tutorial focuses on option (1), which is the most relevant from a practical point of view. We can simply initialize an instance of the `CoulombPotential` class that contains all the necessary functions (such as those defining the short-range and long-range splits) for this potential and makes them useable in the rest of the code.

In [2]:
potential = CoulombPotential()

# Tuning: Find Optimal Hyperparameters
Ewald and mesh methods require the specification of multiple hyperparameters, which are...for non-experts: provide this functionality

In [25]:
...

Ellipsis

# 

# Neighbor List
do not want to compute NL multiple times during training (+ additional subtleties for differentiation) --> compute neighbor list in advance

In [26]:
...

Ellipsis

# Initialize Calculator
explain inputs for both Ewald (for small structures) and P3M (for large structures)

In [27]:
...

Ellipsis

# Compute Energy

In [28]:
...

Ellipsis

# Compute Forces using backpropagation (automatic differentiation)

In [29]:
...

Ellipsis

# Aperiodic Structures

In [30]:
...

Ellipsis

# References to more advanced tasks